# 📘 Treinamento do Modelo – Classificação Single-Label de Gêneros de Jogos

Este notebook realiza o processamento dos dados textuais, preparação do corpus, vetorização com **TF-IDF** e treinamento de um classificador **Single-Label** para prever o gênero principal de um jogo usando sua descrição textual.

## 🔹 Importações
Nesta seção importamos as bibliotecas necessárias para manipulação dos dados, vetorização, treinamento e avaliação do modelo.

In [112]:
import pandas as pd
import ast

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.utils import resample
import joblib
import pathlib

print("Bibliotecas importadas.")


Bibliotecas importadas.


## 🔹 Carregamento do Dataset
Aqui carregamos o arquivo CSV gerado na etapa anterior de coleta de dados e garantimos que não haja valores nulos na descrição.

In [113]:
df = pd.read_csv("../data/games_extended.csv")
print("Shape carregado:", df.shape)

df["description"] = df["description"].fillna("")
df.head()


Shape carregado: (400, 3)


,name,description,genres
0,Grand Theft Auto V,"Rockstar Games went bigger, since their previo...",['Strategy']
1,The Witcher 3: Wild Hunt,"The third game in a series, it holds nothing b...","['Action', 'RPG']"
2,Portal 2,Portal 2 is a first-person puzzle game develop...,"['Shooter', 'Puzzle']"
3,Counter-Strike: Global Offensive,Counter-Strike is a multiplayer phenomenon in ...,['Shooter']
4,Tomb Raider (2013),A cinematic revival of the series in its actio...,['Action']


## 🔹 Seleção de um Único Gênero (primeiro da lista)
O RAWG fornece múltiplos gêneros, mas para este projeto usamos apenas o primeiro gênero, criando assim um problema Single-Label.

In [114]:
def first_genre(genres_obj):
    if isinstance(genres_obj, list):
        return genres_obj[0]
    try:
        genres_list = ast.literal_eval(genres_obj)
        if isinstance(genres_list, list) and len(genres_list) > 0:
            return genres_list[0]
    except Exception:
        pass
    return "Unknown"

df["genre"] = df["genres"].apply(first_genre)

print("Distribuição original dos gêneros:")
print(df["genre"].value_counts())


Distribuição original dos gêneros:
genre
Action        308
Adventure      28
Strategy       20
RPG            18
Shooter        11
Simulation      5
Racing          3
Indie           3
Sports          1
Platformer      1
Casual          1
Arcade          1
Name: count, dtype: int64


## 🔹 Balanceamento das Classes
Para evitar que o modelo aprenda apenas o gênero mais frequente (ex.: Action), realizamos balanceamento via *undersampling*.

In [115]:
counts = df["genre"].value_counts()
print("Distribuição antes do balanceamento:\n", counts)

min_count = counts.min()

df_balanced = pd.concat([
    resample(group, replace=True, n_samples=min_count, random_state=42)
    for genre, group in df.groupby("genre")
])

df = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print("\nDistribuição após o balanceamento:")
print(df["genre"].value_counts())


Distribuição antes do balanceamento:
 genre
Action        308
Adventure      28
Strategy       20
RPG            18
Shooter        11
Simulation      5
Racing          3
Indie           3
Sports          1
Platformer      1
Casual          1
Arcade          1
Name: count, dtype: int64

Distribuição após o balanceamento:
genre
Sports        1
Simulation    1
Action        1
Shooter       1
Platformer    1
Arcade        1
Adventure     1
Strategy      1
Indie         1
Racing        1
Casual        1
RPG           1
Name: count, dtype: int64


## 🔹 Separação Treino/Teste
Dividimos o data set em 80% treino e 20% teste.

In [116]:
X = df["description"]
y = df["genre"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Tamanho treino:", len(X_train))
print("Tamanho teste:", len(X_test))


Tamanho treino: 9
Tamanho teste: 3


## 🔹 Vetorização TF-IDF
Transformamos as descrições em vetores numéricos usando n-grams.

In [117]:
vectorizer = TfidfVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    max_features=20000
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print("Formato vetores treino:", X_train_vec.shape)
print("Formato vetores teste:", X_test_vec.shape)


Formato vetores treino: (9, 2725)
Formato vetores teste: (3, 2725)


## 🔹 Treinamento do Classificador
Usamos um modelo LinearSVC com ajuste de pesos para lidar com classes desbalanceadas.

In [118]:
clf = LinearSVC(class_weight="balanced")
clf.fit(X_train_vec, y_train)
print("Treinamento concluído.")


Treinamento concluído.


## 🔹 Avaliação do Modelo
Aqui exibimos métricas como precision, recall e f1-score.

In [119]:
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

      Action       0.00      0.00      0.00       0.0
      Casual       0.00      0.00      0.00       1.0
      Racing       0.00      0.00      0.00       1.0
     Shooter       0.00      0.00      0.00       0.0
      Sports       0.00      0.00      0.00       1.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



c:\Users\cttcl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\cttcl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\cttcl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

## 🔹 Salvando o Modelo e o Vetorizador
Salvamos os artefatos para que possam ser utilizados no app Streamlit.

In [120]:
model_dir = pathlib.Path("../model")
model_dir.mkdir(exist_ok=True)

joblib.dump(clf, model_dir / "classifier.pkl")
joblib.dump(vectorizer, model_dir / "vectorizer.pkl")

print("Classificador salvo em:", model_dir / "classifier.pkl")
print("Vetorizador salvo em:", model_dir / "vectorizer.pkl")


Classificador salvo em: ..\model\classifier.pkl
Vetorizador salvo em: ..\model\vectorizer.pkl
